In [ ]:
!pip install mediapipe numpy
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.8 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import numpy as np
import scipy
import os
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/mlp"

Mounted at /content/drive


In [ ]:
# Helper function to calculate the angle between three points.
def calculate_angle(a, b, c):
    """
    Calculates the angle (in degrees) between the line segments ab and bc.

    Args:
        a, b, c (list or array): [x, y] coordinates.

    Returns:
        float: The calculated angle in degrees.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Determine which arm to use based on the z-coordinates of the shoulders.
def determine_arm(frame, pose_model):
    """
    Determines which arm to use for analysis (left or right) by comparing
    the z-coordinates of the left and right shoulders.

    Args:
        frame (numpy.ndarray): A single video frame in BGR format.
        pose_model: An instance of the Mediapipe Pose model.

    Returns:
        bool: True if the right arm should be used, False for the left.
    """
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_model.process(image_rgb)
    use_right_arm = True  # Default to right

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        if landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER].z < \
           landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER].z:
            use_right_arm = False

    return use_right_arm

# Process video frames and extract the elbow angles along with timestamps
# and annotated frames.
def process_video_frames(video_path):
    """
    Processes video frames to extract the elbow angle using Mediapipe.

    Args:
        video_path (str): Path to the input video.

    Returns:
        timestamps (np.array): Array of time stamps.
        angles (np.array): Array of computed elbow angles.
        annotated_frames (list): List of tuples (timestamp, frame) after drawing.
        fps (int): Frames per second in the input video.
        frame_width (int): Width of the video frames.
        frame_height (int): Height of the video frames.
    """
    mp_pose = mp.solutions.pose
    pose_model = mp_pose.Pose()

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Read the first frame to decide which arm to track.
    ret, first_frame = cap.read()
    if not ret:
        cap.release()
        pose_model.close()
        raise ValueError("Unable to read video frame")

    use_right_arm = determine_arm(first_frame, pose_model)
    shoulder_landmark = (mp_pose.PoseLandmark.RIGHT_SHOULDER if use_right_arm
                         else mp_pose.PoseLandmark.LEFT_SHOULDER)
    elbow_landmark = (mp_pose.PoseLandmark.RIGHT_ELBOW if use_right_arm
                      else mp_pose.PoseLandmark.LEFT_ELBOW)
    wrist_landmark = (mp_pose.PoseLandmark.RIGHT_WRIST if use_right_arm
                      else mp_pose.PoseLandmark.LEFT_WRIST)

    # Reset to the beginning of the video.
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    angles = []
    timestamps = []
    annotated_frames = []
    frame_idx = 0
    print("Processing video frames...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        timestamp = frame_idx / fps
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[shoulder_landmark.value].x * frame_width,
                        landmarks[shoulder_landmark.value].y * frame_height]
            elbow = [landmarks[elbow_landmark.value].x * frame_width,
                     landmarks[elbow_landmark.value].y * frame_height]
            wrist = [landmarks[wrist_landmark.value].x * frame_width,
                     landmarks[wrist_landmark.value].y * frame_height]

            angle = calculate_angle(shoulder, elbow, wrist)
            angles.append(angle)
            timestamps.append(timestamp)

            # Draw keypoints.
            cv2.circle(frame, tuple(map(int, shoulder)), 8, (0, 255, 0), -1)
            cv2.circle(frame, tuple(map(int, elbow)), 8, (255, 0, 0), -1)
            cv2.circle(frame, tuple(map(int, wrist)), 8, (0, 0, 255), -1)

            # Draw connecting lines.
            cv2.line(frame, tuple(map(int, shoulder)),
                     tuple(map(int, elbow)), (255, 255, 255), 2)
            cv2.line(frame, tuple(map(int, elbow)),
                     tuple(map(int, wrist)), (255, 255, 255), 2)

            # Display the elbow angle.
            cv2.putText(frame, f"Angle: {int(angle)} deg",
                        (int(elbow[0]) + 20, int(elbow[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        annotated_frames.append((timestamp, frame))
        print(f"\rProgress: {(frame_idx/total_frames)*100:.1f}%", end="")

    print("\n")
    cap.release()
    pose_model.close()

    return np.array(timestamps), np.array(angles), annotated_frames, fps, frame_width, frame_height


from scipy.signal import find_peaks

def detect_reps(angles, timestamps, sigma=10, min_distance=50, min_valley_diff=60):
    """
    Detects bicep curl repetitions by identifying peaks (high points) and valleys
    (low points) dynamically.

    Args:
        angles (list or np.array): The elbow angles over time.
        timestamps (list or np.array): The corresponding timestamps.
        sigma (float): Smoothing factor for the angles.
        min_distance (int): Minimum distance between peaks/valleys (in samples).
        min_valley_diff (int): Minimum angle difference required between peak and valley.

    Returns:
        rep_segments (list): List of tuples (start_time, end_time) for each rep.
        angles_smoothed (np.array): The smoothed angle data.
    """
    # Smooth the angles to reduce noise
    angles_smoothed = scipy.ndimage.gaussian_filter1d(angles, sigma=sigma)

    # Find peaks (local maxima) and valleys (local minima)
    peaks, _ = find_peaks(angles_smoothed, distance=min_distance, height=90)  # Only peaks above 90 degrees
    valleys, _ = find_peaks(-angles_smoothed, distance=min_distance, height=(-90))  # Inverted signal for valleys, must be below 90

    # Validate peaks and valleys
    valid_peaks = []
    valid_valleys = []
    for i in range(len(peaks)-1):
        # Find valleys between consecutive peaks
        valley_candidates = [v for v in valleys if peaks[i] < v < peaks[i+1]]

        if valley_candidates:
            # If multiple valleys exist, keep only the one with highest angle (shallowest valley)
            valley = max(valley_candidates, key=lambda v: angles_smoothed[v])

            # Check if valley is deep enough compared to both peaks
            peak1_to_valley = angles_smoothed[peaks[i]] - angles_smoothed[valley]
            valley_to_peak2 = angles_smoothed[peaks[i+1]] - angles_smoothed[valley]

            if peak1_to_valley >= min_valley_diff and valley_to_peak2 >= min_valley_diff:
                valid_peaks.append(peaks[i])
                valid_valleys.append(valley)
                if i == len(peaks)-2:  # Add the last peak if its valley was valid
                    valid_peaks.append(peaks[i+1])

    # Create rep segments from valid peaks
    rep_segments = []
    if valid_peaks:
        # First segment starts from beginning to first valid peak
        rep_segments.append((timestamps[0], timestamps[valid_peaks[0]]))

        # Middle segments
        for i in range(len(valid_peaks)-1):
            rep_segments.append((timestamps[valid_peaks[i]], timestamps[valid_peaks[i+1]]))

        # Last segment from last valid peak to end
        rep_segments.append((timestamps[valid_peaks[-1]], timestamps[-1]))

    return rep_segments

# Plot the elbow angle versus time and mark the detected repetitions.
def plot_elbow_angle(timestamps, angles_smoothed, rep_segments, file_name):
    """
    Plots the smoothed elbow angle over time and highlights the periods
    considered as repetitions.

    Args:
        timestamps (np.array): Array of timestamps.
        angles_smoothed (np.array): Smoothed elbow angle values.
        rep_segments (list): List of rep segments as (start_time, end_time).
    """
    plt.figure(figsize=(15, 5))
    plt.plot(timestamps, angles_smoothed, 'b-', label='Elbow Angle')

    colors=['green', 'red']

    # Highlight the periods considered as repetitions
    for i, (start, end) in enumerate(rep_segments):
        plt.axvspan(start, end, color=colors[i % len(colors)], alpha=0.3, label='Rep Period')

    # Add labels and legend
    plt.xlabel('Time (seconds)')
    plt.ylabel('Elbow Angle (degrees)')
    plt.title(f'Bicep Curl Analysis - {file_name}')
    plt.legend(['Elbow Angle', 'Rep Period'], loc='upper right')
    plt.grid(True)
    plt.gca().invert_yaxis()  # Reverse the y-axis
    plt.show()

# Overlay repetition markers on the annotated video frames.
def overlay_rep_markers(annotated_frames, rep_segments, fps):
    """
    Overlays rep start and end markers on each annotated frame.

    Args:
        annotated_frames (list): List of tuples (timestamp, frame).
        rep_segments (list): List of rep segments as (start_time, end_time).
        fps (int): Frames per second.

    Returns:
        List of frames with overlay annotations.
    """
    updated_frames = []
    for timestamp, frame in annotated_frames:
        # Find current rep
        current_rep = 0
        for i, (start, end) in enumerate(rep_segments):
            if start <= timestamp <= end:
                current_rep = i + 1
            if abs(timestamp - start) < 1 / fps:
                cv2.circle(frame, (50, 50), 15, (0, 255, 0), -1)
                cv2.putText(frame, f"Start Rep {i+1}", (70, 55),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            if abs(timestamp - end) < 1 / fps:
                cv2.circle(frame, (50, 100), 15, (0, 0, 255), -1)
                cv2.putText(frame, f"End Rep {i+1}", (70, 105),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Draw stats overlay
        y_offset = 30
        cv2.putText(frame, f"Time: {timestamp:.1f}s", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 25
        cv2.putText(frame, f"Current Rep: {current_rep}", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 25
        cv2.putText(frame, f"Total Reps: {len(rep_segments)}", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        if current_rep > 0 and current_rep <= len(rep_segments):
            start, end = rep_segments[current_rep-1]
            y_offset += 25
            cv2.putText(frame, f"Rep Duration: {end-start:.1f}s", (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        updated_frames.append(frame)
    return updated_frames

# Write the annotated frames to a video file.
def write_video(frames, output_path, fps, frame_width, frame_height):
    """
    Writes a list of frames to a video file.

    Args:
        frames (list): List of frames (images).
        output_path (str): Path to save the output video.
        fps (int): Frames per second.
        frame_width (int): Width of video frames.
        frame_height (int): Height of video frames.
    """
    out = cv2.VideoWriter(
        output_path,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (frame_width, frame_height)
    )
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Processed video saved as `{output_path}`")

# Main routine that ties everything together.
def run_bicep_curl_analysis(video_path, output_path="bicep_curl_debug.mp4"):
    """
    Orchestrates the bicep curl analysis pipeline:
      1. Process video frames to extract angles.
      2. Detect reps via signal processing.
      3. Plot the elbow angle versus time.
      4. Overlay rep markers on the video.
      5. Write the annotated video to disk.

    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the annotated output video.

    Returns:
        list: List of rep segments (each as (start_time, end_time)).
    """
    start_time = time.time()

    timestamps, angles, annotated_frames, fps, frame_width, frame_height = \
        process_video_frames(video_path)
    rep_segments = detect_reps(angles, timestamps)

    print("\nDetected reps:")
    for i, (start, end) in enumerate(rep_segments):
        print(f"Rep {i+1}: Start = {start:.2f}s, End = {end:.2f}s, "
              f"Duration = {end - start:.2f}s")

    plot_elbow_angle(timestamps, angles, rep_segments, video_path)
    frames_with_overlay = overlay_rep_markers(annotated_frames, rep_segments, fps)
    write_video(frames_with_overlay, output_path, fps, frame_width, frame_height)

    end_time = time.time()
    print(f"\nTotal processing time: {end_time - start_time:.2f} seconds")

    return rep_segments

In [ ]:
# # timestamps, angles, annotated_frames, fps, frame_width, frame_height = \
# #         process_video_frames("./data/12r (1).mp4")

# files = os.listdir("./data")
# print(files)

# # for each file in ./data folder, run the analyze_bicep_curls function
# analysis_results = []
# for file in files:
#     if file.endswith(".mp4"):
#         print(f"Analyzing {file}")
#         timestamps, angles, annotated_frames, fps, frame_width, frame_height = \
#             process_video_frames("./data/" + file)
#         analysis_results.append((file, timestamps, angles, fps, frame_width, frame_height))




In [ ]:
# for file, timestamps, angles, fps, frame_width, frame_height in analysis_results:
#     rep_segments = detect_reps(angles, timestamps)
#     plot_elbow_angle(timestamps, angles, rep_segments, "./data/" + file)


In [ ]:
# files = os.listdir("./data")
# print(files)

# # for each file in ./data folder, run the analyze_bicep_curls function
# for file in files:
#     if file.endswith(".mp4") and "t" in file:
#         print(f"Analyzing {file}")
#         run_bicep_curl_analysis(f"./data/{file}", f"./out/{file}")



In [ ]:
def extract_rep_joint_data(video_path, normalize=True):
    """
    Extracts joint coordinate data for each repetition from a video.

    Args:
        video_path (str): Path to the input video
        normalize (bool): Whether to normalize coordinates relative to shoulder position

    Returns:
        list: List of repetitions, where each rep is a numpy array of shape
             (timesteps, n_features). Features are [elbow_x, elbow_y, wrist_x, wrist_y]
             or their normalized versions relative to shoulder position.
    """
    # Process video to get landmarks
    mp_pose = mp.solutions.pose
    pose_model = mp_pose.Pose()

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Determine which arm to use
    ret, first_frame = cap.read()
    if not ret:
        cap.release()
        pose_model.close()
        raise ValueError("Unable to read video frame")

    use_right_arm = determine_arm(first_frame, pose_model)
    shoulder_landmark = (mp_pose.PoseLandmark.RIGHT_SHOULDER if use_right_arm
                        else mp_pose.PoseLandmark.LEFT_SHOULDER)
    elbow_landmark = (mp_pose.PoseLandmark.RIGHT_ELBOW if use_right_arm
                      else mp_pose.PoseLandmark.LEFT_ELBOW)
    wrist_landmark = (mp_pose.PoseLandmark.RIGHT_WRIST if use_right_arm
                      else mp_pose.PoseLandmark.LEFT_WRIST)

    # Reset video
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Extract coordinates and angles
    coords = []
    angles = []
    timestamps = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        timestamp = frame_idx / fps
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            shoulder = np.array([
                landmarks[shoulder_landmark.value].x * frame_width,
                landmarks[shoulder_landmark.value].y * frame_height
            ])
            elbow = np.array([
                landmarks[elbow_landmark.value].x * frame_width,
                landmarks[elbow_landmark.value].y * frame_height
            ])
            wrist = np.array([
                landmarks[wrist_landmark.value].x * frame_width,
                landmarks[wrist_landmark.value].y * frame_height
            ])

            if normalize:
                # Normalize coordinates relative to shoulder position
                elbow = elbow - shoulder
                wrist = wrist - shoulder

            # Store coordinates as [elbow_x, elbow_y, wrist_x, wrist_y]
            coords.append(np.concatenate([elbow, wrist]))

            angle = calculate_angle(shoulder, elbow + shoulder, wrist + shoulder if normalize else wrist)
            angles.append(angle)
            timestamps.append(timestamp)

    cap.release()
    pose_model.close()

    # Convert to numpy arrays
    coords = np.array(coords)
    angles = np.array(angles)
    timestamps = np.array(timestamps)

    # Detect repetitions
    rep_segments = detect_reps(angles, timestamps)

    # Split coordinates into repetitions
    reps_data = []
    for start_time, end_time in rep_segments:
        start_idx = np.argmin(np.abs(timestamps - start_time))
        end_idx = np.argmin(np.abs(timestamps - end_time))
        rep_coords = coords[start_idx:end_idx+1]
        reps_data.append(rep_coords)

    return reps_data



In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:

def process_all_videos(data_dir):
    """
    Process all videos in a directory and return their rep data.

    Args:
        data_dir (str): Path to directory containing videos

    Returns:
        dict: Dictionary mapping filenames to lists of rep data arrays
    """
    all_sets_data = {}

    for file in os.listdir(data_dir)[1:]:
        if file.endswith('.mp4'):
            print(f"Processing {file}...")
            video_path = os.path.join(data_dir, file)
            sets_data = extract_rep_joint_data(video_path)
            all_sets_data[file] = sets_data
            print(f"Found {len(sets_data)} reps in {file}")
    return all_sets_data

    # Get the first video file
    # files = [f for f in os.listdir(data_dir) if f.endswith('.mp4')]
    # if files:
    #     first_file = files[0]
    #     print(f"Processing {first_file}...")
    #     video_path = os.path.join(data_dir, first_file)
    #     reps_data = extract_rep_joint_data(video_path)

    #     # Print info about the extracted data
    #     print(f"\nFound {len(reps_data)} reps in {first_file}")
    #     for i, rep in enumerate(reps_data):
    #         print(f"Rep {i+1} shape: {rep.shape}")
    # else:
    #     print("No video files found in ./data directory")

    # return reps_data



In [ ]:
def save_video_reps(video_name, reps_data, save_dir='processed_data_npzs'):
    """
    Save rep data for a single video to a .npz file.

    Args:
        video_name (str): Name of the video file
        reps_data (list): List of numpy arrays containing rep data
        save_dir (str): Directory to save the .npz files
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, video_name.replace('.mp4', '.npz'))
    save_dict = {f'rep_{i}': rep for i, rep in enumerate(reps_data)}
    np.savez_compressed(save_path, **save_dict)

def process_videos_directory(data_dir, save_dir='processed_npzs'):
    """
    Process all videos in directory, skipping those that already have .npz files.

    Args:
        data_dir (str): Directory containing videos
        save_dir (str): Directory to save processed data

    Returns:
        dict: Dictionary of processed videos and their rep data
    """
    os.makedirs(save_dir, exist_ok=True)
    all_sets_data = {}
    print(len(os.listdir(data_dir)))

    for file in os.listdir(data_dir):
        if file.endswith('.mp4') or file.endswith('.mov'):
            if file.endswith('.mp4'):
              npz_path = os.path.join(save_dir, file.replace('.mp4', '.npz'))
            else:
              npz_path = os.path.join(save_dir, file.replace('.mov', '.mov.npz'))

            if os.path.exists(npz_path):
                print(f"Skipping {file} - already processed")
                continue

            print(f"Processing {file}...")
            video_path = os.path.join(data_dir, file)
            set_data = extract_rep_joint_data(video_path)
            all_sets_data[file] = set_data

            # Save the processed data
            save_video_reps(file, set_data, save_dir)
            print(f"Found and saved {len(set_data)} reps for {file}")

    return all_sets_data

def save_existing_reps_data(all_reps_data, save_dir='processed_npzs'):
    """
    Save any rep data currently in memory that hasn't been saved yet.

    Args:
        all_reps_data (dict): Dictionary mapping video names to rep data
        save_dir (str): Directory to save processed data
    """
    os.makedirs(save_dir, exist_ok=True)

    for video_name, set_data in all_reps_data.items():
        npz_path = os.path.join(save_dir, video_name.replace('.mp4', '.npz'))
        if not os.path.exists(npz_path):
            print(f"Saving data for {video_name}...")
            save_video_reps(video_name, set_data, save_dir)

import numpy as np

def load_all_processed_data(processed_dir='processed_npzs'):
    """
    Load all processed rep data from .npz files in directory.

    Returns:
        dict: Dictionary mapping video names to a list of rep arrays.
              Each rep array should ideally have shape (timesteps, 4).
    """
    all_sets_data = {}

    for file in os.listdir(processed_dir):
        if file.endswith('.npz'):
            video_name = file.replace('.npz', '.mp4')
            npz_path = os.path.join(processed_dir, file)

            loaded = np.load(npz_path)

            # Let's store arrays in a list
            reps = []
            for key in sorted(loaded.files):
                arr = loaded[key]

                # Debug info: check array type and shape
                print(f"{file} -> key={key}, arr.shape={arr.shape}, arr.dtype={arr.dtype}")

                reps.append(arr)

            all_sets_data[video_name] = reps
            print(f"Loaded {len(reps)} reps from {video_name}")

    return all_sets_data


In [ ]:
all_sets_data = process_videos_directory("./data")

59
Skipping 4_r.mp4 - already processed
Skipping t5.mp4 - already processed
Skipping t1.mp4 - already processed
Skipping t2.mp4 - already processed
Skipping 5_l.mp4 - already processed
Skipping 14r (1).mp4 - already processed
Skipping 11r (1).mp4 - already processed
Skipping 12r (1).mp4 - already processed
Skipping 5_r.mp4 - already processed
Skipping 10r (1).mp4 - already processed
Skipping 8_r (1).mp4 - already processed
Skipping t4.mp4 - already processed
Skipping 6_l.mp4 - already processed
Skipping t6.mp4 - already processed
Skipping 9r (1).mp4 - already processed
Skipping 7_r (1).mp4 - already processed
Skipping t3.mp4 - already processed
Skipping 13r (1).mp4 - already processed
processed_npzs/IMG_5711.mov.npz
Skipping IMG_5711.mov - already processed
processed_npzs/IMG_5712.mov.npz
Skipping IMG_5712.mov - already processed
processed_npzs/IMG_5714.mov.npz
Skipping IMG_5714.mov - already processed
processed_npzs/IMG_5713.mov.npz
Skipping IMG_5713.mov - already processed
processed_